# OSM tags filter

**QuackOSM** allows users to filter the data from the `*.osm.pbf` file. Filtering will reduce a number of features parsed from the original file.

This notebook will explain how to use the OSM tags filtering mechanism.

## Filter format

Library expects a filter in the `dict` form (or `JSON` if provided via CLI).

**QuackOSM** uses two formats of filters: `OsmTagsFilter` and `GroupedOsmTagsFilter`.

In [ ]:
from quackosm._osm_tags_filters import GroupedOsmTagsFilter, OsmTagsFilter

The first one, `OsmTagsFilter`, is a basic `dict` object that defines how to filter OSM based on their tags.

It is based on the filter object used in the [OSMnx](https://osmnx.readthedocs.io/en/stable/index.html) library, but it has more functionalities.

In [ ]:
OsmTagsFilter

The key of the `dict` is expected to be an OSM tag key and the value can be one of: `bool`, a single OSM tag value or a list of OSM tag values.

In [ ]:
# amenity=bench
filter_1 = {"amenity": "bench"}

# amenity=ice_cream and amenity=cafe
filter_2 = {"amenity": ["ice_cream", "cafe"]}

# all amenities
filter_3 = {"amenity": True}

# amenity=bar and building=office
filter_4 = {"amenity": "bar", "building": "office"}

# all amenities and all highways
filter_5 = {"amenity": True, "highway": True}

Second object, `GroupedOsmTagsFilter`, allows assigning filters to groups. It is a `dict` object with a group name being a `key` and `OsmTagsFilter` being a value.

This can become useful for grouping features into semantical categories for machine learning applications.

In [ ]:
GroupedOsmTagsFilter

In [ ]:
# benches
grouped_filter_1 = {"benches": {"amenity": "bench"}}

# swimming sport facilities
grouped_filter_2 = {"swimming_sport": {"leisure": "swimming_pool", "sport": "swimming"}}

# shops, tourism and traffic related objects
grouped_filter_3 = {
    "shopping": {"shop": True, "landuse": "retail"},
    "tourism": {"tourism": True, "historic": True},
    "traffic": {"amenity": "parking", "highway": True},
}

## Basic usage

Examples below show how to use the basic OSM tags filters.

In [ ]:
import urllib.request

from quackosm import get_features_gdf

In [ ]:
monaco_pbf_url = "https://download.geofabrik.de/europe/monaco-latest.osm.pbf"
monaco_pbf_file = "monaco.osm.pbf"
urllib.request.urlretrieve(monaco_pbf_url, monaco_pbf_file)

### Benches only

In [ ]:
tags_filter = {"amenity": "bench"}
get_features_gdf(monaco_pbf_file, tags_filter=tags_filter, silent_mode=True)

### Cafes, bars and restaurants

In [ ]:
tags_filter = {"amenity": ["cafe", "restaurant", "bar"]}
get_features_gdf(monaco_pbf_file, tags_filter=tags_filter, silent_mode=True)

### All amenities and leisures

In [ ]:
tags_filter = {"amenity": True, "leisure": True}
get_features_gdf(monaco_pbf_file, tags_filter=tags_filter, silent_mode=True)

### Shopping and tourism related objects (grouped filters)

In [ ]:
grouped_tags_filter = {
    "shopping": {"shop": True, "landuse": "retail"},
    "tourism": {"tourism": True, "historic": True},
}
get_features_gdf(monaco_pbf_file, tags_filter=grouped_tags_filter, silent_mode=True)

## Compact and exploded tags

**QuackOSM** has the option to operate in two distinct modes when it comes to the result `GeoParquet` file schema:
- The first one keeps all loaded OSM tags as a `dict` object under the `tag` column name. This format can simplify storage schema (just 3 columns) and is recommended for a big number of loaded osm tags (or when there is no `tags_filter` applied).
- The second one generates one column per each OSM tag key (or per group name when using `GroupedOsmTagsFilter`). This mode can simplify further operations in the geospatial analyses.

Section below explains the difference between them and how they interact with other available parameters.

**Parameters logic table.**

The table shows how the columns for the result are generated based on value of the `explode_tags`, `keep_all_tags` parameters with and without OSM tags filter being present.

Legend:

- ✔️ - `True`
- ❌ - `False`
- 📦 - Compact tags (single `tags` column)
- 💥 - Exploded tags (separate columns per each tag key, or group name)

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;margin:0px auto;}
.tg td{border-style:solid;border-width:1px;font-size:1em;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-style:solid;border-width:1px;font-size:1em;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0so2{font-family:inherit;text-align:center;vertical-align:middle}
.tg .tg-header{background-color:var(--jp-rendermime-table-row-background);}
.tg .tg-highlight{background-color:var(--jp-rendermime-table-row-hover-background);}
@media screen and (max-width: 767px) {.tg {width: auto !important;}.tg col {width: auto !important;}.tg-wrap {overflow-x: auto;-webkit-overflow-scrolling: touch;margin: auto 0px;}}</style>
<div class="tg-wrap"><table class="tg">
<thead>
  <tr class="tg-header">
    <th class="tg-0so2"><strong>OSM filter</strong><sup>1</sup></th>
    <th class="tg-0so2"><strong><code>keep_all_tags</code></strong><sup>2</sup></th>
    <th class="tg-0so2"><strong><code>explode_tags</code></strong></th>
    <th class="tg-0so2"><strong>Resulting columns</strong></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2"><code>None</code></td>
    <td class="tg-0so2">📦</td>
  </tr>
  <tr>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">💥</td>
  </tr>
  <tr>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2">📦</td>
  </tr>
  <tr class="tg-highlight">
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2"><code>None</code></td>
    <td class="tg-0so2">💥<sup>3</sup></td>
  </tr>
  <tr>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">💥</td>
  </tr>
  <tr>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2">📦</td>
  </tr>
  <tr class="tg-highlight">
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2"><code>N/A</code></td>
    <td class="tg-0so2"><code>None</code></td>
    <td class="tg-0so2">📦<sup>4</sup></td>
  </tr>
  <tr>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2"><code>N/A</code></td>
    <td class="tg-0so2">✔️</td>
    <td class="tg-0so2">💥</td>
  </tr>
  <tr>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2"><code>N/A</code></td>
    <td class="tg-0so2">❌</td>
    <td class="tg-0so2">📦</td>
  </tr>
</tbody>
</table></div><br/>

`1` - refers to the `tags_filter` parameter: ✔️ - filter is provided, ❌ - filter is empty<br/>
`2` - `keep_all_tags` is used only when `tags_filter` is provided (`N/A` - not applicable) <br/>
`3` - default parameter values with `tags_filter` <br/>
`4` - default parameter values without `tags_filter`


Examples below show how to use `explode_tags` and `keep_all_tags` parameters.

In [ ]:
tags_filter = {"amenity": "library"}

### Default mode with OSM tags filter

Will load only OSM tags from the provided filter. <br/>
Default parameter values: `explode_tags`=✔️, `keep_all_tags`=❌

In [ ]:
get_features_gdf(monaco_pbf_file, tags_filter=tags_filter, silent_mode=True).sort_index()

### Default mode without OSM tags filter

Load all features from the PBF file. <br/>
Default parameter values: `explode_tags`=❌, `keep_all_tags`=`N/A` (takes all OSM tags, since there is no filter)

In [ ]:
get_features_gdf(monaco_pbf_file, tags_filter=None, silent_mode=True).sort_index()

### 1. `explode_tags`=✔️, `keep_all_tags`=❌

Keep only tags from the provided `tags_filter` and save them as separate columns.

In [ ]:
get_features_gdf(
    monaco_pbf_file,
    tags_filter=tags_filter,
    explode_tags=True,
    keep_all_tags=False,
    silent_mode=True,
).sort_index()

### 2. `explode_tags`=✔️, `keep_all_tags`=✔️

Keep all OSM tags and save them as separate columns.

In [ ]:
get_features_gdf(
    monaco_pbf_file,
    tags_filter=tags_filter,
    explode_tags=True,
    keep_all_tags=True,
    silent_mode=True,
).sort_index()

### 3. `explode_tags`=❌, `keep_all_tags`=❌

Keep only tags from the provided `tags_filter` and save them as a single `tags` column.

In [ ]:
get_features_gdf(
    monaco_pbf_file,
    tags_filter=tags_filter,
    explode_tags=False,
    keep_all_tags=False,
    silent_mode=True,
).sort_index()

### 4. `explode_tags`=❌, `keep_all_tags`=✔️

Keep all OSM tags and save them as a single `tags` column.

In [ ]:
get_features_gdf(
    monaco_pbf_file,
    tags_filter=tags_filter,
    explode_tags=False,
    keep_all_tags=True,
    silent_mode=True,
).sort_index()

## Positive and negative filters

## Wildcard filters

## Invalid filters